In [1]:
%scala
import org.apache.spark.sql.functions.{col,to_date}

val df= Seq(
   ("Alex", "04/01/1992"),
   ("Sarah","01/01/2000"),
   ("David","01/01/2005")
  ).toDF("name","Birthdate").withColumn("Birthdate", to_date(col("Birthdate"),"dd/MM/yyyy")).show()

+-----+----------+
 name| Birthdate|
+-----+----------+
 Alex|1992-01-04|
Sarah|2000-01-01|
David|2005-01-01|
+-----+----------+

import org.apache.spark.sql.functions.{col, to_date}
df: Unit = ()

In [2]:


list=[['Enviar solicitud',1209],
      ['Analizar mapa de',12155],
    ['Entregar servicio',91049],
     ['example1',1900],
     ['example2', 2500],
     ['example3', 9282398],
     ['example4',14000]]

df=spark.createDataFrame(list, ['Name', 'Seconds'])
df.show()

+-----------------+-------+
 Name|Seconds|
+-----------------+-------+
 Enviar solicitud| 1209|
 Analizar mapa de| 12155|
Entregar servicio| 91049|
 example1| 1900|
 example2| 2500|
 example3|9282398|
 example4| 14000|
+-----------------+-------+

In [3]:
from pyspark.sql import functions as F
from pyspark.sql.functions import when
df.withColumn("Minutes", F.round((F.col("Seconds")/60),2))\
.withColumn("Hours", F.floor((F.col("Minutes")/60)))\
.withColumn("hourmin", F.floor(F.col("Minutes")-(F.col("Hours").cast("int") * 60)))\
.withColumn("Days", F.floor((F.col("Hours")/24)))\
.withColumn("Days2", F.col("Days")*24)\
.withColumn("Time", F.when((F.col("Hours")==0) &(F.col("Days")==0), F.concat(F.col("hourmin"),F.lit("min"))).when((F.col("Hours")!=0)&(F.col("Days")==0), F.concat(F.col("Hours"),F.lit("hr "),F.col("hourmin"),F.lit("min"))).when(F.col("Days")!=0, F.concat(F.col("Days"),F.lit("d "),(F.col("Hours")-F.col("Days2")),F.lit("hr "),F.col("hourmin"),F.lit("min"))))\
.drop("Minutes","Hours","hourmin","Days","Days2")\
.show()

+-----------------+-------+---------------+
 Name|Seconds| Time|
+-----------------+-------+---------------+
 Enviar solicitud| 1209| 20min|
 Analizar mapa de| 12155| 3hr 22min|
Entregar servicio| 91049| 1d 1hr 17min|
 example1| 1900| 31min|
 example2| 2500| 41min|
 example3|9282398|107d 10hr 26min|
 example4| 14000| 3hr 53min|
+-----------------+-------+---------------+

In [4]:
df1.printSchema()

root
-- Name: string (nullable = true)
-- Seconds: long (nullable = true)
-- Time: string (nullable = true)

In [5]:
from pyspark.sql import functions as F
from pyspark.sql.types import *

list=[['1h:23m'],
     ['23m']]
df=spark.createDataFrame(list,['time'])
df.show()



+------+
 time|
+------+
1h:23m|
 23m|
+------+

In [6]:
df.withColumn("seconds", F.when(F.col("time").contains("h"), F.unix_timestamp(F.regexp_replace("time", "h|m", ''),"H:mm"))\
.otherwise(F.unix_timestamp(F.substring("time",1,2),"mm")))\
.show()

+------+-------+
 time|seconds|
+------+-------+
1h:23m| 4980|
 23m| 1380|
+------+-------+

In [7]:
unix_timestamp(col("time"), "H:mm")